In [ ]:
import itertools
import torch
import torch.optim as optim
from tensorflow import keras
from PIL import Image
from torchvision import models, transforms
import torch.nn as nn
import random
from numpy.linalg import norm
import numpy as np
import math
import time
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
import sys
root = '../../../'
sys.path.append(root)
from HelpfulFunctions.batchCreation import createBatch
from HelpfulFunctions.metrics import meanAveragePrecision
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def CreateDataset(root, num_classes, batch_size, train = 1):
    if train == 1:
        #Create X_train_tensor
        X_train = np.load( root + "Features/train_features_vgg16_cifar10.npy" ) # Shape = (45000, 4096)
        X_train_tensor = torch.tensor(X_train)

        #Create Y_train_tensor
        y_train = np.load( root + "Features/train_labels_vgg16_cifar10.npy" ) # Shape = (45000,)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        y_train_tensor = torch.nn.functional.one_hot(y_train_tensor, num_classes) #One-Hot Encoded -> Shape = (45000, num_classes)

        #Create indices
        indices_train = torch.arange(len(X_train_tensor))

        dataset = TensorDataset(X_train_tensor, y_train_tensor, indices_train)
        train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return train_loader

    elif train == 2:
        X_validation = np.load( root + "Features/validation_features_vgg16_cifar10.npy" ) # Shape = (10000, 4096)
        X_validation_tensor = torch.tensor(X_validation)

        y_validation = np.load( root + "Features/validation_labels_vgg16_cifar10.npy" ) # Shape = (10000,)
        y_validation_tensor = torch.tensor(y_validation, dtype=torch.long)
        y_validation_tensor = torch.nn.functional.one_hot(y_validation_tensor, num_classes) #One-Hot Encoded -> Shape = (10000, num_classes)

        #Create indices
        indices_validation = torch.arange(len(X_validation_tensor))

        dataset = TensorDataset(X_validation_tensor, y_validation_tensor, indices_validation)
        validation_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return validation_loader

    elif train == 3:
        X_test = np.load( root + "Features/test_features_vgg16_cifar10.npy" ) # Shape = (10000, 4096)
        X_test_tensor = torch.tensor(X_test)

        y_test = np.load( root + "Features/test_labels_vgg16_cifar10.npy" ) # Shape = (10000,)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long)
        y_test_tensor = torch.nn.functional.one_hot(y_test_tensor, num_classes) #One-Hot Encoded -> Shape = (10000, num_classes)

        #Create indices
        indices_test = torch.arange(len(X_test_tensor))

        dataset = TensorDataset(X_test_tensor, y_test_tensor, indices_test)
        test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        return test_loader


    

In [ ]:
class CustomNN(nn.Module):
    def __init__(self):
        super(CustomNN, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(4096, 1024),  # First fully connected layer
            nn.ReLU(),
            nn.Linear(1024, 32),    # Second fully connected layer to reduce to 4000
        )

        # Initialize weights and biases from gaussian distribution
        for layer in self.fc_layers:
            if isinstance(layer, nn.Linear):
                nn.init.normal_(layer.weight, mean=0.0, std=0.01)  # Initialize weights based on paper
                nn.init.normal_(layer.bias, mean=0.0, std=0.01)    # Initialize biases based on paper

    def forward(self, x):
        return self.fc_layers(x)

In [ ]:
class DPSHLoss(torch.nn.Module):
    def __init__(self, train_size, n_classes, bit):
        super(DPSHLoss, self).__init__()
        self.U = torch.zeros(train_size, bit).float().to(device)
        self.Y = torch.zeros(train_size, n_classes).float().to(device)

    def forward(self, u, y, ind, eta):
        self.U[ind, :] = u.data
        self.Y[ind, :] = y.float()

        s = (y @ self.Y.t() > 0).float()
        inner_product = u @ self.U.t() * 0.5

        likelihood_loss = (1 + (-(inner_product.abs())).exp()).log() + inner_product.clamp(min=0) - s * inner_product

        likelihood_loss = likelihood_loss.mean()

        quantization_loss = eta * (u - u.sign()).pow(2).mean()

        return likelihood_loss + quantization_loss

In [ ]:
# Define the grid

def DPSH(train_loader: torch.utils.data.DataLoader, validation_loader: torch.utils.data.DataLoader, test_loader: torch.utils.data.DataLoader, device: torch.device, train_size: int, n_classes: int, bit: int, num_epoch: int, batch_size: int, eta_values: list, wd_values: list, lr_values: list):


    train_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = 1)
    test_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = 2)
    validation_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = 3)
    














    param_grid = {
        'eta': eta_values,
        'learning_rate': lr_values,
        #'batch_size': [16, 32, 64],
        'weight_decay': wd_values
    }

    customLoss = DPSHLoss(train_size, n_classes, bit)


    # Get all combinations of parameters
    keys, values = zip(*param_grid.items())
    parameter_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    # Evaluate each parameter combination
    best_params = None
    best_accuracy = 0

    for params in parameter_combinations:
        print(f"Testing combination: {params}")
    
        # Initialize loss function with specific parameters
        loss_fn = customLoss(eta = params['eta'])

    # Initialize model and optimizer
    model = CustomNN().to(device)
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Train the model
    for epoch in range(num_epoch):  # Example epoch count
        model.train()
        train_loss = 0
        for image, label, ind in train_loader:
            image = image.to(device)
            label = label.to(device)

            optimizer.zero_grad()
            u = model(image)

            loss = loss_fn(u, label.float(), ind)
            train_loss += loss.item()

            loss.backward()
            optimizer.step()

        train_loss = train_loss / (train_size / batch_size)
    # Validate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for image, label, ind in validation_loader:
            outputs = model(image)
            predicted = torch.argmax(outputs, dim=1)
            correct += (predicted == label).sum().item()
            total += label.size(0)

    accuracy = correct / total
    print(f"Validation Accuracy: {accuracy:.4f}")

    # Update best parameters
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = params

    print("Best Parameters:", best_params)
    print("Best Accuracy:", best_accuracy)
